In [79]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv('data_cleaned.csv')

In [81]:
# removing columns
df.drop(
    ["education", "marital_status", "Teenhome", "Kidhome", "NumWebVisitsMonth", "MntGoldProds"],
    axis=1,
    inplace=True
)
df.head()

,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Customer_Days
0,58138.0,58,635,88,546,172,88,3,8,10,4,False,False,False,False,False,False,True,63,2822
1,46344.0,38,11,1,6,2,1,2,1,1,2,False,False,False,False,False,False,False,66,2272
2,71613.0,26,426,49,127,111,21,1,8,2,10,False,False,False,False,False,False,False,55,2471
3,26646.0,26,11,4,20,10,3,2,2,0,4,False,False,False,False,False,False,False,36,2298
4,58293.0,94,173,43,118,46,27,5,5,3,6,False,False,False,False,False,False,False,39,2320


In [82]:
# unifying purchases
tot_purchases = np.sum(df.iloc[:, 8:11], axis=1)
df.drop(df.columns[8:11], axis=1, inplace=True)
df.insert(8, "TotPurchases", tot_purchases)
df.head()

,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,NumDealsPurchases,TotPurchases,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Customer_Days
0,58138.0,58,635,88,546,172,88,3,22,False,False,False,False,False,False,True,63,2822
1,46344.0,38,11,1,6,2,1,2,4,False,False,False,False,False,False,False,66,2272
2,71613.0,26,426,49,127,111,21,1,20,False,False,False,False,False,False,False,55,2471
3,26646.0,26,11,4,20,10,3,2,6,False,False,False,False,False,False,False,36,2298
4,58293.0,94,173,43,118,46,27,5,14,False,False,False,False,False,False,False,39,2320


In [83]:
# unifying AccemptedCmps
sorted_cmps = np.hstack((df.iloc[:, 12:14], df.iloc[:, 9:12]))
accepted_cmps_new = np.zeros(df.shape[0], dtype=np.int8)
indexed_accepted_cmps = np.argwhere(sorted_cmps)  # extracting index
# Convert to DataFrame
accepted_cmp_df = pd.DataFrame(indexed_accepted_cmps, columns=['index', 'col'])
# unifying by min
accepted_cmps_index = (accepted_cmp_df.groupby('index', as_index=False)['col'].min()).to_numpy() # using the min cause first campaigns are more important
# assigning val
for i, index in enumerate(accepted_cmps_index[:, 0]):
    accepted_cmps_new[index] = accepted_cmps_index[i, 1] + 1
# modifying df
df.drop(df.columns[9:14], axis=1, inplace=True)
df.insert(9, "AcceptedCmps", accepted_cmps_new)
df.head()

,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,NumDealsPurchases,TotPurchases,AcceptedCmps,Complain,Response,Age,Customer_Days
0,58138.0,58,635,88,546,172,88,3,22,0,False,True,63,2822
1,46344.0,38,11,1,6,2,1,2,4,0,False,False,66,2272
2,71613.0,26,426,49,127,111,21,1,20,0,False,False,55,2471
3,26646.0,26,11,4,20,10,3,2,6,0,False,False,36,2298
4,58293.0,94,173,43,118,46,27,5,14,0,False,False,39,2320


In [84]:
# saving the dataset
df.to_csv("./data_simplified.csv", index=False)